### This notebook consist of code for creating the html files for the website each time data is updated.

### Set-up

Importing classes

In [28]:
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path

Defining paths

In [130]:
GROUP_DATA_DIR = Path("../..//group-data")
MEMBERS_DIR_PATH = GROUP_DATA_DIR / "members/"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / "content"
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "templates"
HOSTING_PATH = GROUP_DATA_DIR.parent / "kerzendorf-group.github.io"

Data Processing Parameters

In [106]:
# List of JSON files to be processed
JSON_FILES_TO_LOAD = [
    "general",
    "homepage",
    "research",
    "support",
    "contact",
]
# Needed columns for homepage
ARTICLE_METADATA_FIELDS = [
    "article_id",
    "category",
    "date",
    "tags",
    "title",
    "cover_image",
    "short_description",
]
# Groups and institution used in filtering data
GROUP_FILTER = ["DTI", "TARDIS", "ICER", "kerzendorf"]
INSTITUTION_FILTER = "Michigan State University"

# Map roles to standardized roles for consistency
ROLE_MAP = {
    "Assistant Professor": "Professor",
    "Professor": "Professor",
    "Visualization Consultant": "Visualization Consultant",
    "Research Consultant": "Research Consultant",
    "Research Software Engineer": "Research Software Engineer",
    "Professorial Assistant": "Undergraduate",
    "Visiting Researcher": "Postdoctoral Researcher",
    "Postdoctoral Researcher": "Postdoctoral Researcher",
}

# Map degrees to standardized academic levels
DEGREE_MAP = {
    "Masters": "Graduate Student",
    "PhD": "Postdoctorate",  #  if end_date is present
    "Bachelors": "Graduate Student",
}

Function to create proper HTML file names by replacing spaces with underscores

In [107]:
def page_link(a):
    """Return the HTML file name after replacing blank spaces(" ") with underscores("-")"""
    return a.replace(" ", "_") if " " in a else a

 Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates

In [133]:
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH), extensions=["jinja2.ext.loopcontrols"]
)
environment.globals["page_link"] = page_link

Function for looping through JSON files and loading their content into the 'data' dictionary 

In [2]:
def loading_json_files(json_data_list):
    """
    Load data from JSON files specified in a list of file names.

    Parameters:
    ----------
    json_data_list : list of str
        A list of file names (without extension) to load as JSON.

    Returns:
    -------
    dict
        A dictionary where keys are file names and values are the corresponding JSON data.

    Raises:
    ------
    FileNotFoundError:
        If a specified file does not exist.
    json.JSONDecodeError:
        If there's an issue decoding the JSON content from a file.

    """
    data = {}
    for data_id in json_data_list:
        data_path = WEBSITE_DATA_PATH / f"{data_id}.json"

        try:
            with open(data_path, "r") as json_file:
                data[json_file] = json.load(json_file)
        except FileNotFoundError:
            print(f"File '{json_file}.json' not found.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON in '{json_file}.json'.")
    return data

Function to create a page

In [110]:
def create_page(template, html, **kwargs):
    """
    Create an HTML page using a Jinja2 template and save it to a specified path.

    Parameters:
    ----------
    template : str
        The filename of the Jinja2 template to be used.
    html : str
        The filename of the HTML file to be generated.
    **kwargs : dict
        Additional keyword arguments to be passed to the Jinja2 template for rendering.

    Returns:
    -------
    None

    """
    page_template = environment.get_template(template)
    page_html_path = HOSTING_PATH / html
    page_content = page_template.render(**kwargs)
    with open(page_html_path, mode="w", encoding="utf-8") as page:
        page.write(page_content)

Creating dataframes for articles which can be updated further 

In [111]:
def load_content_from_files(columns):
    """
    Load content data from JSON files into a Pandas DataFrame.

    Parameters
    ----------
    columns : list of str
        A list of column names to extract from the JSON files.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the specified columns from the loaded JSON files.

    """
    content_data = {col: [] for col in columns}

    for json_file in os.listdir(CONTENT_DIR_PATH):
        if json_file.endswith(".json"):
            json_path = os.path.join(CONTENT_DIR_PATH, json_file)
            with open(json_path, "r") as file:
                info = json.load(file)
                # Only load those articles where display is True
                if info.get("display"):
                    for col in columns:
                        content_data[col].append(info.get(col))

    content_df = pd.DataFrame(content_data)
    content_df["date"] = pd.to_datetime(content_df["date"], format="%m-%d-%Y")
    return content_df

Extract the Latest Content for Each Category from a DataFrame

In [112]:
def get_latest_content_df(content_df):
    """
    Extract the latest content for each category from a DataFrame.

    Parameters
    ----------
    content_df : pandas.DataFrame
        The input DataFrame containing content information.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the latest content for each category.

    """
    # Sort the entire DataFrame by "category" and "date" in descending order
    sorted_content_df = content_df.sort_values(
        by=["category", "date"], ascending=[True, False]
    )

    # Get the first row for each category using groupby and head
    latest_content_df = sorted_content_df.groupby("category").head(1).copy()
    latest_content_df["date"] = pd.to_datetime(
        latest_content_df["date"], format="%m-%d-%Y"
    )
    latest_content_df = latest_content_df.sort_values(by="date", ascending=False)

    return latest_content_df

### Processing List Of JSON files

In [113]:
# Function Call
data = loading_json_files(JSON_FILES_TO_LOAD)

### Homepage

Storing selected columns for Homepage only

In [114]:
content_df = load_content_from_files(ARTICLE_METADATA_FIELDS)
latest_content_df = get_latest_content_df(content_df)

In [115]:
create_page(
    "homepage.html.j2",
    "Index.html",
    general=data["general"],
    homepage=data["homepage"],
    recent_content=latest_content_df.to_dict(orient="records"),
)

### People Page

Filtering based on group and institution

In [116]:
def filter_edu_exp_data(df, valid_groups,valid_institution):
    """
    Filter education and experience data based on specified criteria.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing education and experience data.
    valid_groups : list
        List of valid groups to include in the filtered data.
    valid_institution : str
        The valid institution to include in the filtered data.

    Returns
    -------
    pandas.DataFrame
        A filtered DataFrame containing only the rows that meet the specified criteria.
    """
    group_mask = False
    institution_mask = False

    # Check if 'group' column exists and update mask accordingly
    if "group" in df.columns:
        group_mask = df["group"].isin(valid_groups)

    # Check if 'institution' column exists and update mask accordingly
    if "institution" in df.columns:
        institution_mask = df["institution"] == valid_institution

    final_mask = group_mask | institution_mask
    return df[final_mask]

Function to load education data

In [3]:
def load_education_experience_data(directory):
    """
    Load education and experience data from JSON files, filter based on criteria, and perform preprocessing.

    Parameters
    ----------
    directory : str or pathlib.Path
        The directory path containing "experiences.json" and "education.json" files.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing filtered and preprocessed education and experience data.
    """
    filtered_records = []
    file_names = ["experiences.json", "education.json"]
    for file_name in file_names:
        file_path = directory / file_name
        if file_path.exists():
            # Reading JSON data directly into a DataFrame
            records = pd.read_json(file_path)

            # filtering based on group and institution
            valid_records = filter_edu_exp_data(records, GROUP_FILTER, INSTITUTION_FILTER)

            filtered_records.append(valid_records)
        else:
            print(f"{file_path} does not exist")

    if filtered_records:
        combined_records = pd.concat(filtered_records, ignore_index=True)
    else:
        combined_records = pd.DataFrame()

    # if start_date column exists, fill with NaN if it doesn't
    if "start_date" not in combined_records:
        combined_records["start_date"] = pd.NaT

    # Convert start_date to datetime format
    combined_records["start_date"] = pd.to_datetime(combined_records["start_date"], errors="coerce")

    # Sort the DataFrame based on start_date
    combined_records = combined_records.sort_values(by="start_date", ascending=False)
    return combined_records

Function load social links directly

In [4]:
def load_social_links(social_dir):
    """
    Load social links from a JSON file.

    Parameters
    ----------
    social_dir : str or pathlib.Path
        The directory path containing the "social_links.json" file.

    Returns
    -------
    dict or None
        A dictionary containing social links or None if the file doesn't exist.
    """
    social_links = None
    social_links_file_path = social_dir / "social_links.json"
    if social_links_file_path.exists():
        with open(social_links_file_path, "r") as f:
            social_links = json.load(f)
    return social_links

Function to load topmost project title

In [5]:
def load_latest_project_title(project_dir):
    """
    Load the title of the topmost project from a JSON file.

    Parameters
    ----------
    project_dir : str or pathlib.Path
        The directory path containing the "projects.json" file.

    Returns
    -------
    str or None
        The title of the topmost project or None if the file doesn't exist or is empty.
    """
    projects_file_path = project_dir / "projects.json"
    topmost_project_title = None
    if projects_file_path.exists():
        projects_df = pd.read_json(projects_file_path)
        if not projects_df.empty:
            # Fetching the project title from the first row of the DataFrame
            topmost_project_title = projects_df.iloc[0].get("project_title")
    return topmost_project_title

Funtion to parse member data

In [6]:
def parse_member_data(member_dir):
    """
    Parse member-related data from JSON files in the specified directory.

    Parameters
    ----------
    member_dir : str or pathlib.Path
        The directory path containing member-related JSON files.

    Returns
    -------
    tuple
        A tuple containing education and experience DataFrame, social links dictionary,
        and the title of the current project.
    """
    member_json_dir = member_dir / "jsons"
    education_experience_df = load_education_experience_data(member_json_dir)
    current_project_title = load_latest_project_title(member_json_dir)
    social_links = load_social_links(member_json_dir)

    return education_experience_df, social_links, current_project_title

In [7]:
# Function to extract academic roles from education and experience data
def extract_member_academic_role(education_experience_df):
    """
    Extract the current academic role of a member based on education and experience data.

    Parameters
    ----------
    education_experience_df : pandas.DataFrame
        DataFrame containing education and experience data.

    Returns
    -------
    Tuple[str, bool]
        A tuple containing:
        - str: The current academic role of the member.
        - bool: True if the member is currently active, False otherwise.
    """

    # Check if these columns exist in dataframe
    for column in ["end_date", "group", "institution"]:
        if column not in education_experience_df.columns:
            education_experience_df[column] = None

    current_academic_role = None

    for _, row in education_experience_df.iterrows():
        role = row.get("role", None)
        degree = row.get("degree", None)

        if not current_academic_role:
            current_academic_role = ROLE_MAP.get(role, "")

            if degree == "PhD" and pd.isna(row["end_date"]):
                current_academic_role = "Graduate Student"  # if end_date is NaN
            elif degree == "Bachelors" and pd.isna(row["end_date"]):
                current_academic_role = "Undergraduate Student"
            elif not current_academic_role and degree in DEGREE_MAP:
                current_academic_role = DEGREE_MAP[degree]

    # Check for end dates outside the loop
    has_end_date = all(
        not pd.isna(date) for date in education_experience_df["end_date"]
    )
    is_current_member = not has_end_date

    return current_academic_role, is_current_member

In [8]:
# Lists to store data for current and alumni members
def fetch_member_data():
    """
    Fetch and process member data from directories in the specified members' directory.

    Returns
    -------
    Tuple[list, list]
        A tuple containing two lists:
        1. List of dictionaries representing current members' data.
        2. List of dictionaries representing alumni members' data.

    """
    current_people_page_list = []
    alumni_people_page_list = []
    # Looping through member directories to fetch and process member data
    for member_dir in MEMBERS_DIR_PATH.glob("*"):
        if not (member_info_fname := member_dir / "info.json").exists():
            continue
        else:
            member_info = json.load(open(member_info_fname, "r"))
        (
            education_experience_df,
            social_links,
            current_project_title,
        ) = parse_member_data(member_dir)
        current_academic_role, is_current_member = extract_member_academic_role(
            education_experience_df
        )
        first_name = member_info["first_name"]
        last_name = member_info["last_name"]
        nickname = member_info.get("nick_name", None)
        id = member_info["id"]
        image_path = member_info["image_path"]
        cover_image_path = member_info["cover_image_path"]

        name = f"{nickname if nickname else first_name} {last_name}"

        member_data = {
            "name": name,
            "academic_role": current_academic_role,
            "id": id,
            "current_project_title": current_project_title,
            "image_path": image_path,
            "cover_image_path": cover_image_path,
        }

        if social_links is not None:
            member_data.update(social_links)

        if is_current_member:
            current_people_page_list.append(member_data)
        else:
            alumni_people_page_list.append(member_data)

    return current_people_page_list, alumni_people_page_list

In [9]:
current_people_page_list, alumni_people_page_list = fetch_member_data()

NameError: name 'MEMBERS_DIR_PATH' is not defined

In [10]:
create_page(
    "people.html.j2",
    "People.html",
    general=data["general"],
    current_members=current_people_page_list,
    alumni_members=alumni_people_page_list,
)

NameError: name 'create_page' is not defined

### Contact Page

In [125]:
create_page(
    "contact.html.j2", "Contact.html", general=data["general"], contact=data["contact"]
)

### Support Page

In [126]:
create_page(
    "support.html.j2", "Support.html", general=data["general"], support=data["support"]
)

### Research Front Page

For adding more columns in dataframe to render front pages and individual article pages

In [127]:
columns_extended = ARTICLE_METADATA_FIELDS + ["author_id"]
content_df = load_content_from_files(columns_extended)
research_content_df = content_df[content_df["category"] != "News"].sort_values(
    by=["category", "date"], ascending=[True, False]
)
latest_content_df = get_latest_content_df(content_df)

In [128]:
create_page(
    "research.html.j2",
    "Research.html",
    general=data["general"],
    content=research_content_df,
    research=data["research"],
    current_members=current_people_page_list,
)

In [129]:
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")


for category in content_df.loc[content_df.category != "News", "category"].unique():
    sub_research_content = sub_research_template.render(
        general=data["general"],
        research=data["research"],
        content=latest_content_df,
        category=category,
        current_members=current_people_page_list,
    )
    folder_path = f"{HOSTING_PATH}/sub_research/{page_link(category.lower())}"
    os.makedirs(folder_path, exist_ok=True)
    with open(f"{folder_path}.html", mode="w", encoding="utf-8") as sub_research:
        sub_research.write(sub_research_content)